In [1]:
# Core Python
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

# ML / DL
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# API / HTTP
import requests

# -----------------------------
# Dummy ADK classes for testing
# -----------------------------
class Tool:
    def __init__(self, name, function):
        self.name = name
        self.function = function

class Memory:
    def __init__(self):
        self.store_data = []
    def store(self, item):
        self.store_data.append(item)
    def retrieve_all(self):
        return self.store_data

class Agent:
    def __init__(self, name, description=""):
        self.name = name
        self.description = description
        self.tools = {}
        self.memory = Memory()
    def add_tool(self, tool):
        self.tools[tool.name] = tool
    def use_tool(self, tool_name, *args):
        if tool_name in self.tools:
            return self.tools[tool_name].function(*args)
        else:
            return f"Tool {tool_name} not found."

print("Dummy ADK classes loaded. You can now continue development and testing.")

2025-11-24 13:49:00.178460: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763992140.419368      73 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763992140.494211      73 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Dummy ADK classes loaded. You can now continue development and testing.


In [2]:
# Assuming your medicinal plant dataset is organized as:
# dataset/
#    class_1/
#    class_2/
#    ...
dataset_path = "/kaggle/input/medicinal-leaves-for-hybridvit-and-vgg19/Medicinal_leaf_augmented_dataset"

# List classes
classes = os.listdir(dataset_path)
print("Classes:", classes)

# Count images per class
for cls in classes:
    print(cls, ":", len(os.listdir(os.path.join(dataset_path, cls))))

Classes: ['Nageshor', 'Ginger', 'Pathorkuchi', 'Debdaru', 'Cordia', 'Tejpata', 'Neem', 'Jarul', 'Ashok', 'Telakucha', 'Minjiri', 'Cannonball_Tree', 'Vasaka', 'Holud_Pata', 'Mastwood', 'Royna', 'Akondo', 'Punnag', 'Basil', 'Thankuni']
Nageshor : 1000
Ginger : 1000
Pathorkuchi : 1000
Debdaru : 1000
Cordia : 1000
Tejpata : 1000
Neem : 1000
Jarul : 1000
Ashok : 1000
Telakucha : 1000
Minjiri : 1000
Cannonball_Tree : 1000
Vasaka : 1000
Holud_Pata : 1000
Mastwood : 1000
Royna : 1000
Akondo : 1000
Punnag : 1000
Basil : 1000
Thankuni : 1000


In [3]:
# Image size for CNN input
IMG_SIZE = (224, 224)

# ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=32,
    subset="training",
    class_mode="categorical"
)

val_gen = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=32,
    subset="validation",
    class_mode="categorical"
)


Found 16000 images belonging to 20 classes.
Found 4000 images belonging to 20 classes.


In [4]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(classes), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1763992172.419752      73 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1763992172.420580      73 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    23,888,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 20)             │         2,580 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,909,972 (91.21 MB)

 Trainable params: 23,909,972 (91.21 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    verbose=1
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


I0000 00:00:1763992177.635369     121 service.cc:148] XLA service 0x78d8a40052e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1763992177.636230     121 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1763992177.636253     121 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1763992177.960677     121 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/500 ━━━━━━━━━━━━━━━━━━━━ 1:03:25 8s/step - accuracy: 0.0312 - loss: 3.0541

I0000 00:00:1763992182.821869     121 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


500/500 ━━━━━━━━━━━━━━━━━━━━ 367s 719ms/step - accuracy: 0.1813 - loss: 3.1095 - val_accuracy: 0.4487 - val_loss: 1.7985
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 283s 566ms/step - accuracy: 0.4929 - loss: 1.5436 - val_accuracy: 0.6273 - val_loss: 1.0777
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 284s 569ms/step - accuracy: 0.6203 - loss: 1.1316 - val_accuracy: 0.7320 - val_loss: 0.8199
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 285s 570ms/step - accuracy: 0.6895 - loss: 0.9002 - val_accuracy: 0.7525 - val_loss: 0.7864
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 295s 591ms/step - accuracy: 0.7409 - loss: 0.7566 - val_accuracy: 0.7818 - val_loss: 0.6387
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 299s 597ms/step - accuracy: 0.7512 - loss: 0.7303 - val_accuracy: 0.8092 - val_loss: 0.6021
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 288s 576ms/step - accuracy: 0.7875 - loss: 0.6108 - val_accuracy: 0.8198 - val_loss: 0.5736
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 281s 561ms/step - accuracy: 0.8067 - loss: 0.56

In [6]:
model.save("medicinal_plant_classifier.h5")

In [7]:
from tensorflow.keras.models import load_model
classifier_model = load_model("medicinal_plant_classifier.h5")

In [8]:
def classify_plant(image_path):
    img = Image.open(image_path).resize(IMG_SIZE)
    img_array = np.expand_dims(np.array(img)/255.0, axis=0)
    pred = classifier_model.predict(img_array)
    class_idx = np.argmax(pred)
    class_label = train_gen.class_indices
    class_label = list(class_label.keys())[list(class_label.values()).index(class_idx)]
    return class_label


In [9]:
# Example: Plant info API (replace with real API or dataset)
def get_plant_info(plant_name):
    # Mockup example; replace with actual API if available
    info_db = {
        "/kaggle/input/medicinal-leaves-for-hybridvit-and-vgg19/Medicinal_leaf_augmented_dataset",
       
    }
    return info_db.get(plant_name, "Information not available.")

In [10]:
# Create your agent
plant_agent = Agent(
    name="MedicinalPlantAdvisor",
    description="Identifies medicinal plants and provides usage & care information."
)

# Add tools
plant_agent.add_tool(Tool(name="Classify Plant", function=classify_plant))
plant_agent.add_tool(Tool(name="Plant Knowledge", function=get_plant_info))

# Add memory
plant_agent.memory = Memory()

In [11]:
import random
import os
import numpy as np
from PIL import Image

# -----------------------------
# Assumes your trained classifier model is loaded as `classifier_model`
# and input size is defined
# -----------------------------
IMG_SIZE = (224, 224)  # Adjust based on your model

# -----------------------------
# Update classify_plant to include all 20 classes
# -----------------------------
def classify_plant(image_path):
    img = Image.open(image_path).resize(IMG_SIZE)
    img_array = np.expand_dims(np.array(img)/255.0, axis=0)
    pred = classifier_model.predict(img_array)  # Replace with your trained model
    class_idx = np.argmax(pred)
    
    # Full class names (replace with your actual dataset class names in correct order)
    class_names = [
        "Akondo", "Ashok", "Basil", "Cannonball_Tree", "Cordia", "Debdaru", "Ginger",
        "Holud_pata", "Jarul", "Mastwood", "Minjiri", "Nageshor", "Neem",
        "Pathorkuchi", "Punnag", "Royna", "Tejpata", "Telakucha", "Thankuni", "Vasaka"
    ]
    return class_names[class_idx]

# -----------------------------
# Knowledge retrieval
# -----------------------------
def get_plant_info(plant_name):
    info_db = {
    "Akando": "Akando (Calotropis gigantea or Calotropis procera), also known as Madar or Crown Flower, has extensive medicinal value in traditional systems like Ayurveda, Unani, and Siddha medicine. Nearly every part of the plant—leaves, flowers, roots, and the milky latex—is used for various ailments, primarily due to its anti-inflammatory, analgesic, and antimicrobial properties",
    "Ashok": "Ashoka tree bark is used in traditional medicine for women's reproductive health, especially for menstrual disorders.",
    "Basil": "Basil has anti-inflammatory, antimicrobial, and antioxidant properties and supports digestion and respiratory health.",
    "Cannonball_Tree": "Cannonball tree leaves and flowers are used in traditional medicine for wound healing, skin infections, and anti-inflammatory benefits.",
    "Cordia": "Cordia leaves and fruits are used for treating cough, fever, and digestive disorders.",
    "Debdaru": "Debdaru (Polyalthia longifolia) is used for fever reduction, skin diseases, and anti-inflammatory purposes.",
    "Ginger": "Ginger is widely used for nausea, digestion, inflammation, and respiratory issues.",
    "Holud_pata": "Turmeric leaves contain curcumin-like compounds and are used for anti-inflammatory, antioxidant, and digestive benefits.",
    "Jarul": "Jarul (Queen's Crape Myrtle) leaves are used traditionally for diabetes control and metabolic health.",
    "Mastwood": "Mastwood (Calophyllum inophyllum) oil and leaves are used for skin healing, anti-inflammatory, and antimicrobial purposes.",
    "Minjiri": "Minjiri (Moringa) leaves are rich in nutrients and used for immunity, inflammation reduction, and metabolic health.",
    "Nageshor": "Nageshor (Mesua ferrea) flowers and seeds are used for cardiovascular health, anti-inflammatory, and digestive support.",
    "Neem": "Neem has strong antibacterial, antifungal, antiviral, and anti-inflammatory properties used for skin, blood purification, and infections.",
    "Pathorkuchi": "Pathorkuchi (Kalanchoe) leaves are used for kidney stones, wound healing, and anti-inflammatory purposes.",
    "Punnag": "Punnag (Calophyllum) seeds and oil are used for skin disorders, inflammation, and wound healing.",
    "Royna": "Royna (Amoora rohituka) is used in Ayurveda for liver health, tumors, and digestive disorders.",
    "Tejpata": "Tejpata (Bay leaf) is used for digestion, blood sugar control, and antimicrobial benefits.",
    "Telakucha": "Telakucha (Coccinia) leaves are used for diabetes management, digestion, and anti-inflammatory effects.",
    "Thankuni": "Thankuni (Centella asiatica) is used for memory enhancement, wound healing, anxiety relief, and skin health.",
    "Vasaka": "Vasaka (Malabar nut) leaves are used for respiratory issues like cough, asthma, and bronchitis due to strong expectorant properties."
    }
    return info_db.get(plant_name, "Information not available.")

# -----------------------------
# Agent setup (dummy ADK for now)
# -----------------------------
class Tool:
    def __init__(self, name, function):
        self.name = name
        self.function = function

class Memory:
    def __init__(self):
        self.store_data = []
    def store(self, item):
        self.store_data.append(item)
    def retrieve_all(self):
        return self.store_data

class Agent:
    def __init__(self, name, description=""):
        self.name = name
        self.description = description
        self.tools = {}
        self.memory = Memory()
    def add_tool(self, tool):
        self.tools[tool.name] = tool
    def use_tool(self, tool_name, *args):
        if tool_name in self.tools:
            return self.tools[tool_name].function(*args)
        else:
            return f"Tool {tool_name} not found."

plant_agent = Agent(name="MedicinalPlantAgent", description="Identifies medicinal plants and provides info.")
plant_agent.add_tool(Tool("Classify Plant", classify_plant))
plant_agent.add_tool(Tool("Plant Knowledge", get_plant_info))

# -----------------------------
# Random image selection from dataset
# -----------------------------
dataset_folder = "/kaggle/input/medicinal-leaves-for-hybridvit-and-vgg19/Medicinal_leaf_augmented_dataset"

# Pick a random class folder
class_folder = os.path.join(dataset_folder, random.choice(os.listdir(dataset_folder)))

# Pick a random image inside the class folder
image_file = random.choice([f for f in os.listdir(class_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
user_image = os.path.join(class_folder, image_file)

print("Testing image:", user_image)

# -----------------------------
# Run agent
# -----------------------------
plant_name = plant_agent.use_tool("Classify Plant", user_image)
plant_info = plant_agent.use_tool("Plant Knowledge", plant_name)

# Store in memory
plant_agent.memory.store({"query_image": user_image, "plant": plant_name, "info": plant_info})

# -----------------------------
# Display results
# -----------------------------
print("Plant Identified:", plant_name)
print("Info:", plant_info)
print("\nMemory Stored:", plant_agent.memory.retrieve_all())


Testing image: /kaggle/input/medicinal-leaves-for-hybridvit-and-vgg19/Medicinal_leaf_augmented_dataset/Debdaru/_14_2868162.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 961ms/step
Plant Identified: Debdaru
Info: Debdaru (Polyalthia longifolia) is used for fever reduction, skin diseases, and anti-inflammatory purposes.

Memory Stored: [{'query_image': '/kaggle/input/medicinal-leaves-for-hybridvit-and-vgg19/Medicinal_leaf_augmented_dataset/Debdaru/_14_2868162.png', 'plant': 'Debdaru', 'info': 'Debdaru (Polyalthia longifolia) is used for fever reduction, skin diseases, and anti-inflammatory purposes.'}]


In [12]:
# Agent 1: Image Classifier
classifier_agent = Agent(name="ClassifierAgent")
classifier_agent.add_tool(Tool("Classify Plant", classify_plant))

# Agent 2: Knowledge Agent
knowledge_agent = Agent(name="KnowledgeAgent")
knowledge_agent.add_tool(Tool("Plant Info", get_plant_info))

# Agent 3: Advisory Agent (optional)
advisory_agent = Agent(name="AdvisoryAgent")
def advise(plant_name):
    return f"Remember to harvest {plant_name} sustainably!"
advisory_agent.add_tool(Tool("Advise", advise))

# Use A2A Protocol
classifier_result = classifier_agent.use_tool("Classify Plant", user_image)
knowledge_result = knowledge_agent.use_tool("Plant Info", classifier_result)
advisory_result = advisory_agent.use_tool("Advise", classifier_result)

print(classifier_result, knowledge_result, advisory_result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Debdaru Debdaru (Polyalthia longifolia) is used for fever reduction, skin diseases, and anti-inflammatory purposes. Remember to harvest Debdaru sustainably!


In [13]:
# Log user interaction consistently
interaction_log = {
    "image_path": user_image,
    "plant_name": plant_name,
    "plant_info": plant_info
}

# Store in memory
plant_agent.memory.store(interaction_log)

# Display memory logs
print("Memory Logs:", plant_agent.memory.retrieve_all())


Memory Logs: [{'query_image': '/kaggle/input/medicinal-leaves-for-hybridvit-and-vgg19/Medicinal_leaf_augmented_dataset/Debdaru/_14_2868162.png', 'plant': 'Debdaru', 'info': 'Debdaru (Polyalthia longifolia) is used for fever reduction, skin diseases, and anti-inflammatory purposes.'}, {'image_path': '/kaggle/input/medicinal-leaves-for-hybridvit-and-vgg19/Medicinal_leaf_augmented_dataset/Debdaru/_14_2868162.png', 'plant_name': 'Debdaru', 'plant_info': 'Debdaru (Polyalthia longifolia) is used for fever reduction, skin diseases, and anti-inflammatory purposes.'}]


In [14]:
import io
import os
import random
from PIL import Image
import ipywidgets as widgets
from IPython.display import display, Image as IPImage, clear_output

# ----------------------------
# CONFIG: Dataset folder and image size
# ----------------------------
DATASET_FOLDER = "/kaggle/input/medicinal-leaves-for-hybridvit-and-vgg19/Medicinal_leaf_augmented_dataset"
IMG_SIZE = (224, 224)  # example, use your model's input size

# ----------------------------
# Helper function to get a random dataset image
# ----------------------------
def get_random_dataset_image():
    class_folder = os.path.join(DATASET_FOLDER, random.choice(os.listdir(DATASET_FOLDER)))
    image_file = random.choice([f for f in os.listdir(class_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    return os.path.join(class_folder, image_file)

# ----------------------------
# Widgets
# ----------------------------
use_upload = widgets.ToggleButtons(
    options=['Random Dataset Image', 'Upload Image'],
    description='Select Input:',
)
upload = widgets.FileUpload(
    accept='.jpg,.jpeg,.png',
    multiple=False
)
run_button = widgets.Button(description="Classify Plant")
output = widgets.Output()

# Display widgets
display(use_upload, upload, run_button, output)
upload.layout.display = 'none'  # hide initially

# Toggle upload visibility
def toggle_upload(change):
    upload.layout.display = 'block' if change['new'] == 'Upload Image' else 'none'
use_upload.observe(toggle_upload, names='value')

# ----------------------------
# Main function
# ----------------------------
def process_image(b):
    with output:
        clear_output()
        
        # Step 1: Get image path
        if use_upload.value == 'Upload Image' and len(upload.value) > 0:
            # For Kaggle, upload.value is a tuple, not a dict
            uploaded_file = upload.value[0]  # first uploaded file
            image_bytes = uploaded_file['content']
            user_image = Image.open(io.BytesIO(image_bytes))
            
            # Save locally for the model
            user_image_path = "/kaggle/working/uploaded_image.png"
            user_image.save(user_image_path)
            
            # Display uploaded image
            display(IPImage(filename=user_image_path))
        else:
            # Random dataset image
            user_image_path = get_random_dataset_image()
            user_image = Image.open(user_image_path)
            display(IPImage(filename=user_image_path))
        
        # Step 2: Classification
        plant_name = plant_agent.use_tool("Classify Plant", user_image_path)

        # Step 3: Knowledge Retrieval
        plant_info = plant_agent.use_tool("Plant Knowledge", plant_name)

        # Step 4: Store in memory
        plant_agent.memory.store({
            "image_path": user_image_path,
            "plant_name": plant_name,
            "plant_info": plant_info
        })

        # Step 5: Display results
        print("\n--- Plant Classification Result ---")
        print("Plant Identified:", plant_name)
        print("Info:", plant_info)
        print("\n--- Memory Logs ---")
        print(plant_agent.memory.retrieve_all())

# Connect button click
run_button.on_click(process_image)


ToggleButtons(description='Select Input:', options=('Random Dataset Image', 'Upload Image'), value='Random Dat…

FileUpload(value=(), accept='.jpg,.jpeg,.png', description='Upload')

Button(description='Classify Plant', style=ButtonStyle())

Output()